# Set up

In [1]:
import numpy
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

data = pd.read_csv('/kaggle/input/frases-playlist/frases_playlist.csv')

In [2]:
data.head()

,frase
0,Quiero una playlist que me motive en el gimnasio.
1,"Me ha dejado mi pareja, quiero una playlist pa..."
2,"Voy a hacer un viaje largo, quiero una playlis..."
3,No se qué canciones poner en el coche.
4,Quiero música para meditar.


# Create dataset

In [3]:
!pip install openai
from openai import AzureOpenAI

AZURE_OPENAI_API_KEY = "7MoKzuuPAla0Cl3RYSM2aHjTAmkVopeVC4VhHCCdLPGDbAg0Vk4OJQQJ99AKACYeBjFXJ3w3AAABACOGrjrr"
API_VERSION = "2024-10-01-preview"
API_ENDPOINT = "https://practicas-2024.openai.azure.com/"
API_LOCATION = "eastus"

API_MODEL_MINI = "gpt-4o-mini"
API_MODEL = "gpt-4o"

client = AzureOpenAI(
            api_version=API_VERSION,
            azure_endpoint=API_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 7.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 14.7 MB/s eta 0:00:00


In [4]:
CONTEXT="""
Por la frase que recibas debes etiquetarla con:
Pop
Rock
Hip-Hop
EDM
Indie
Jazz
Soul
Reggaetón
Música Clásica
Folk
R&B
Canciones acústicas
Lo-fi
Música instrumental
Música ambiental
Música latina
Funk
Música tropical
Bossa Nova
Dance

Separa las etiquetas con coma
"""

def receive_message(text):
        try:
            chat_completion = client.chat.completions.create(
                model=API_MODEL,
                messages=[
                    {"role": "system", "content": CONTEXT},
                    {"role": "user", "content": text},
                ],
                max_tokens=10,
                temperature=0.0,  # Reducir la aleatoriedad para respuestas consistentes
            )

            response = chat_completion.choices[0].message.content.strip()
            return response

        except:
            return "ERROR"

In [5]:
data['genero']=data['frase'].progress_apply(receive_message)

100%|██████████| 1005/1005 [11:16<00:00,  1.49it/s]


In [6]:
data.head()

,frase,genero
0,Quiero una playlist que me motive en el gimnasio.,"Pop, EDM, Hip-Hop, Dance, Reg"
1,"Me ha dejado mi pareja, quiero una playlist pa...","Pop, R&B, Indie, Canciones acúst"
2,"Voy a hacer un viaje largo, quiero una playlis...","Música ambiental, Lo-fi, Música instrumental,"
3,No se qué canciones poner en el coche.,"Pop, Rock, Hip-Hop, EDM, Indie"
4,Quiero música para meditar.,"Música ambiental, Música instrumental, Lo-fi"


In [7]:
genders = ["Pop", "Rock","Hip-Hop","EDM","Indie","Jazz","Soul","Reggaetón","Música Clásica","Folk","R&B","Canciones acústicas","Lo-fi",
"Música instrumental","Música ambiental","Música latina","Funk","Música tropical","Bossa Nova","Dance"]

In [8]:
for gender in genders:
    data[gender] = data['genero'].apply(lambda frase: gender in frase)

In [9]:
data.head()

,frase,genero,Pop,Rock,Hip-Hop,EDM,Indie,Jazz,Soul,Reggaetón,...,R&B,Canciones acústicas,Lo-fi,Música instrumental,Música ambiental,Música latina,Funk,Música tropical,Bossa Nova,Dance
0,Quiero una playlist que me motive en el gimnasio.,"Pop, EDM, Hip-Hop, Dance, Reg",True,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,"Me ha dejado mi pareja, quiero una playlist pa...","Pop, R&B, Indie, Canciones acúst",True,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,"Voy a hacer un viaje largo, quiero una playlis...","Música ambiental, Lo-fi, Música instrumental,",False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False
3,No se qué canciones poner en el coche.,"Pop, Rock, Hip-Hop, EDM, Indie",True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Quiero música para meditar.,"Música ambiental, Música instrumental, Lo-fi",False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False


# Try embbeding

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

case = 2
if case == 1:
    vectorizer = TfidfVectorizer()
elif case == 2:
    vectorizer = CountVectorizer(stop_words=stopwords.words('spanish'))
elif case == 3:
    vectorizer = CountVectorizer(stop_words=stopwords.words('spanish'), ngram_range=(1,2))

X = vectorizer.fit_transform(data['frase'])
y = data.drop(['frase','genero'], axis=1)

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Try models

In [86]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

case = 1
if case == 1:
    model = LogisticRegression(solver='liblinear', max_iter=1000) #Accuracy media: 0.910945273631841
elif case == 2:
    model = SVC(kernel='linear', probability=False) #Accuracy media: 0.909452736318408
elif case == 3:
    model = RandomForestClassifier(n_estimators=200, random_state=42) #Accuracy media: 0.9042288557213931
elif case == 4:
    model = DecisionTreeClassifier(max_depth=5, random_state=42) #Accuracy media: 0.8952736318407961
elif case == 5:
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42) #Accuracy media: 0.9017412935323383
elif case == 6:
    model = KNeighborsClassifier(n_neighbors=5) #Accuracy media: 0.883582089552239


multi_target_model = MultiOutputClassifier(model)

multi_target_model.fit(X_train, y_train)

y_pred = multi_target_model.predict(X_test)


In [87]:
from sklearn.metrics import accuracy_score

med = 0
for i, genre in enumerate(genders):  
    acc = accuracy_score(y_test.iloc[:, i], y_pred[:, i]) 
    med += acc
    print(f"Accuracy para {genre}: {acc}")

med /= len(genders)
print(f"Accuracy media: {med}")

Accuracy para Pop: 0.8557213930348259
Accuracy para Rock: 0.9054726368159204
Accuracy para Hip-Hop: 0.9502487562189055
Accuracy para EDM: 0.9104477611940298
Accuracy para Indie: 0.8407960199004975
Accuracy para Jazz: 0.8756218905472637
Accuracy para Soul: 0.9552238805970149
Accuracy para Reggaetón: 0.9601990049751243
Accuracy para Música Clásica: 0.9253731343283582
Accuracy para Folk: 0.9651741293532339
Accuracy para R&B: 0.9253731343283582
Accuracy para Canciones acústicas: 0.945273631840796
Accuracy para Lo-fi: 0.8606965174129353
Accuracy para Música instrumental: 0.8009950248756219
Accuracy para Música ambiental: 0.8159203980099502
Accuracy para Música latina: 0.9353233830845771
Accuracy para Funk: 0.9751243781094527
Accuracy para Música tropical: 0.9601990049751243
Accuracy para Bossa Nova: 0.9552238805970149
Accuracy para Dance: 0.900497512437811
Accuracy media: 0.910945273631841


# Export

In [89]:
import joblib

joblib.dump((multi_target_model, vectorizer), 'musical_gender_model.pkl')

['musical_gender_model.pkl']